# Buildingblocks for programming preconditioners

In [ ]:
import netgen.gui
from netgen.geom2d import unit_square
from ngsolve import *
%gui tk
from ngsolve.la import EigenValues_Preconditioner
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
Draw (mesh)

In [ ]:
fes = H1(mesh, order=3, dirichlet="left|bottom")
u,v = fes.TnT()
a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v) + u*v)
a.Assemble()
f = LinearForm(fes)
f += SymbolicLFI(1*v)
f.Assemble()
gfu = GridFunction(fes)

### Create a Jacobi-preconditioner

In [ ]:
pre = a.mat.CreateSmoother(fes.FreeDofs())

In [ ]:
EigenValues_Preconditioner(mat=a.mat, pre=pre).NumPy()

In [ ]:
solvers.CG(mat=a.mat, pre=pre, rhs=f.vec, sol=gfu.vec)
Draw (gfu)

### The smoother can also perform Gauss-Seidel smoothing:

In [ ]:
gfu.vec[:] = 0
res = f.vec.CreateVector()
pres = f.vec.CreateVector()
proj = Projector(fes.FreeDofs(), True)

for i in range(500):
    pre.Smooth(gfu.vec, f.vec)
    res.data = f.vec - a.mat*gfu.vec
    pres.data = proj * res
    print ("it", i, ", res =", Norm(pres))
Draw (gfu)

### Implement a forward-backward GS preconditioner

In [ ]:
class SymmetricGS(BaseMatrix):
    def __init__ (self, smoother):
        super(SymmetricGS, self).__init__()
        self.smoother = smoother
    def Mult (self, x, y):
        y[:] = 0.0
        self.smoother.Smooth(y, x)
        self.smoother.SmoothBack(y,x)
    def Height (self):
        return self.smoother.height
    def Width (self):
        return self.smoother.height

In [ ]:
gspre = SymmetricGS(pre)
solvers.CG(mat=a.mat, pre=gspre, rhs=f.vec, sol=gfu.vec)
Draw (gfu)

In [ ]:
EigenValues_Preconditioner(mat=a.mat, pre=gspre).NumPy()

## A BlockJacobi

In [ ]:
blocks = []
freedofs = fes.FreeDofs()
for v in mesh.vertices:
    vdofs = set()
    for el in mesh[v].elements:
        vdofs |= set(d for d in fes.GetDofNrs(el) if freedofs[d])
    blocks.append (vdofs)
print (blocks)

In [ ]:
blockjac = a.mat.CreateBlockSmoother(blocks)
EigenValues_Preconditioner(mat=a.mat, pre=blockjac)

In [ ]:
blockgs = SymmetricGS(blockjac)
EigenValues_Preconditioner(mat=a.mat, pre=blockgs)

### Add a coarse grid correction

In [ ]:
vertexdofs = BitArray(fes.ndof)
vertexdofs[:] = False
for v in mesh.vertices:
    for d in fes.GetDofNrs(v):
        vertexdofs[d] = True
vertexdofs &= fes.FreeDofs()
print (vertexdofs)

In [ ]:
coarsepre = a.mat.Inverse(vertexdofs)
EigenValues_Preconditioner(mat=a.mat, pre=coarsepre)

In [ ]:
twogrid = coarsepre + blockgs 
EigenValues_Preconditioner(mat=a.mat, pre=twogrid)